In [1]:
import pandas as pd
import os
import json



In [2]:
headers=["sentence_id", "sentence", "phrase_start", "phrase_end", 'phrase', 'num_native_annotator', "num_non_native_annotator", "num_complex_marks_native", "num_complex_marks_non_native", "complex_binary", "complex_prob"]

datasets=["../data/cwishareddataset/traindevset/english/Wikipedia_Dev.tsv","../data/cwishareddataset/traindevset/english/Wikipedia_Train.tsv", "../data/cwishareddataset/testset/english/Wikipedia_Test.tsv"]
dataset_names = ['Wikipedia_dev', "Wikipedia_train", "Wikipedia_test"]

data = pd.read_csv(datasets[0], sep='\t', names=headers)
data

,sentence_id,sentence,phrase_start,phrase_end,phrase,num_native_annotator,num_non_native_annotator,num_complex_marks_native,num_complex_marks_non_native,complex_binary,complex_prob
0,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,4,8,tail,10,10,0,2,1,0.10
1,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,12,25,Epidexipteryx,10,10,6,3,1,0.45
2,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,31,35,bore,10,10,0,3,1,0.15
3,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,31,53,bore unusual vertebrae,10,10,0,1,1,0.05
4,3QI9WAYOGQCX8YMZA9CAS9VCVMWS62,The tail of Epidexipteryx also bore unusual ve...,36,53,unusual vertebrae,10,10,2,0,1,0.10
...,...,...,...,...,...,...,...,...,...,...,...
689,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,100,109,knowledge,10,10,0,2,1,0.10
690,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,74,80,person,10,10,0,0,0,0.00
691,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,95,99,true,10,10,0,0,0,0.00
692,3W0KKJIARRAMOTSFYF06L10TKN9K8Z,Devotion ( Bhakti ) will cancel the effects of...,113,122,purifying,10,10,7,8,1,0.75


In [3]:
def count_words(row):
    phrase = row['phrase']
    return len(phrase.split())

def get_words(data):
    data["complex_prob_str"] = data.apply(lambda row: format(row['complex_prob'], ".2f"), axis=1 )
    data['num_words'] = data.apply(lambda row: count_words(row), axis=1)
    words = data[data['num_words']==1]
    return words
    
words = get_words(data)

In [4]:
words[['phrase', 'complex_prob', 'num_words']].sort_values(['complex_prob'], ascending=False)

,phrase,complex_prob,num_words
217,thermodynamics,1.00,1
427,unearthed,0.95,1
560,prototype,0.95,1
477,monasteries,0.95,1
261,enlargement,0.90,1
...,...,...,...
183,source,0.00,1
411,Birthday,0.00,1
410,Queen,0.00,1
409,Australia,0.00,1


In [5]:
def get_text_and_metadata(words):

    text = " ".join(words['phrase'].tolist())

    phrases = words['phrase'].to_list()
    weights = words['complex_prob'].to_list()
    

    word_weights = words[['phrase', 'complex_prob']]
    
    return text, word_weights


print(get_text_and_metadata(words))



('tail Epidexipteryx bore unusual vertebrae towards resembled tip feather-anchoring pygostyle oviraptorosaurs modern birds Epidexipteryx preserved covering simpler body feathers composed parallel barbs primitive feathered dinosaurs modern-style However rectrices feathers tail vanes individual branched filaments ribbon-like made single sheet maniraptoran reported Jurassic dinosaur Middle age Upper Jurassic Daohugou Beds Inner Mongolia China mya Epidexipteryx genus display feather paravian small dinosaur fossil specimen collection Institute Vertebrate Paleontology Paleoanthropology Beijing Medicine recognizes altitudes metres start ft affect extreme humans record humans living altitudes metres ft years aviation types altitude altitude types various explained simply measuring ways atmosphere altitude Earth divided altitude surface regions Regions Earth atmosphere mean high referred sea level altitude high Aviation measured altitude Mean Sea Level MSL local ground level Above reference Gro

In [29]:
directory = "../data/evaluation/" 

if not os.path.exists(directory):
    os.makedirs(directory)

texts, wordsets = [],[]
    
for name, dataset in zip(dataset_names, datasets):
    data = pd.read_csv(dataset, sep='\t', names=headers)
    words = get_words(data)
    text, wordset = get_text_and_metadata(words)

    texts.append(text)
    wordsets.append(wordset)
    
    with open(directory+name+".txt", "w")as f:
        f.write(text)
        
    wordset.to_csv(path_or_buf=directory+name+".csv")


In [10]:
def evaluate_ranking(reference_df, df_under_test):

    res = reference_df.merge(df_under_test, "outer", left_on="phrase", right_on="token")

    res["ranking_error"] = (
        abs(res["complex_prob"] + res["ranking"]) * res["complex_prob"]
    )

    return res

In [24]:
#from src.utils.evaluation_utils.evaluation import evaluate_ranking
from src.utils.output_utils.create_df_from_doc import create_df_from_doc
from src.pipeline.create_pipeline import create_pipeline

pipeline = create_pipeline()
doc = pipeline(texts[0])
df = create_df_from_doc(doc)


618 overall words


In [26]:
df

,token,lemma,appearance,difficulty,rel. frequency,ranking,cluster,example phrase
259,mellifera,mellifera,1,6.13,35.334979,0.868955,NaN,human rights expression personal freedom imple...
242,species,specie,2,5.54,36.365583,0.846250,NaN,human rights expression personal freedom imple...
113,thermodynamics,thermodynamic,3,5.18,35.650375,0.813916,NaN,literature drama exhibitions art thermodynamic...
96,librettist,librettist,1,5.94,22.768017,0.684293,NaN,founded Festival composer librettist Benjamin ...
152,subdivided,subdivide,1,5.77,15.401894,0.572390,NaN,spectral Notes subdivided classes numerals
...,...,...,...,...,...,...,...,...
21,rectrices,rectrice,1,8.00,NaN,NaN,NaN,Epidexipteryx preserved covering simpler body ...
31,maniraptoran,maniraptoran,1,8.00,NaN,NaN,NaN,individual branched filaments ribbon-like made...
38,paravian,paravian,1,8.00,NaN,NaN,NaN,China mya Epidexipteryx genus display feather ...
204,sekihan,sekihan,1,8.00,NaN,NaN,NaN,Japan particular boiled resulting sugar ingred...


In [22]:
df_res = evaluate_ranking(wordsets[0], df)
df_res
df_res.sort_values(by='complex_prob', axis=0, ascending=False, na_position="last")

,phrase,complex_prob,token,lemma,appearance,difficulty,rel. frequency,ranking,cluster,example phrase,ranking_error
198,thermodynamics,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384,unearthed,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,prototype,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,monasteries,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
515,emissions,0.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
930,NaN,NaN,rectrices,rectrice,1.0,8.0,NaN,NaN,NaN,Epidexipteryx preserved covering simpler body ...,NaN
931,NaN,NaN,maniraptoran,maniraptoran,1.0,8.0,NaN,NaN,NaN,individual branched filaments ribbon-like made...,NaN
932,NaN,NaN,paravian,paravian,1.0,8.0,NaN,NaN,NaN,China mya Epidexipteryx genus display feather ...,NaN
933,NaN,NaN,sekihan,sekihan,1.0,8.0,NaN,NaN,NaN,Japan particular boiled resulting sugar ingred...,NaN
